In [1]:
# import json as json
# import pandas as pd
# import requests as requests
# def getLatLng(address):
#     url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
#     headers = {"Authorization": "KakaoAK 073a393656181c6073880062d3507191"}
#     result = json.loads(str(requests.get(url, headers=headers).text))
#     match_first = result['documents'][0]['address']
#     return float(match_first['y']), float(match_first['x'])
# getLatLng('부산 사직동')

In [2]:
import requests; from urllib.parse import urlparse
import pandas as pd
import geopandas
# 필요한 module다운받기 -> geopandas가 import되지 않는다면? 직접 anaconda prompt에서 설치해야한다
# conda install -c conda-forge geopandas (근데 설치시간이 드릅게 오래 걸린다!)

In [3]:
address = "부산광역시 동래구 사직동 26-9번지"
url = "https://dapi.kakao.com/v2/local/search/address.json?query=" + address
# 카카오맵 주소검색 url에다 address를 붙여 요청
result = requests.get(urlparse(url).geturl(),
                      headers={"Authorization": "KakaoAK 073a393656181c6073880062d3507191"})
json_obj = result.json()
print(json_obj)
# 요청시 나온 결과(result)를 json으로 담아서 print해본 것.
# json으로 담는게 (json_obj = result.json() <- 이 행위) 뭘까????? 
# 그냥 result만 print하면 <Response [200]> 가 뜨고 만다. 띠요용
# 카카오맵 document에 있는 녀석들이 몽땅 나온다. 얘네들은 각 index안에 값이 담긴 list형태다

{'documents': [{'address': {'address_name': '부산 동래구 사직동 26-9', 'b_code': '2626010900', 'h_code': '2626059000', 'main_address_no': '26', 'mountain_yn': 'N', 'region_1depth_name': '부산', 'region_2depth_name': '동래구', 'region_3depth_h_name': '사직2동', 'region_3depth_name': '사직동', 'sub_address_no': '9', 'x': '129.059764973464', 'y': '35.1990136794377'}, 'address_name': '부산 동래구 사직동 26-9', 'address_type': 'REGION_ADDR', 'road_address': {'address_name': '부산 동래구 사직북로 35', 'building_name': '', 'main_building_no': '35', 'region_1depth_name': '부산', 'region_2depth_name': '동래구', 'region_3depth_name': '사직동', 'road_name': '사직북로', 'sub_building_no': '', 'underground_yn': 'N', 'x': '129.059802648053', 'y': '35.1990317823955', 'zone_no': '47860'}, 'x': '129.059764973464', 'y': '35.1990136794377'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}


In [4]:
list = []
for document in json_obj['documents']:
    val = [document['road_address']['building_name'], document['address_name'], document['y'], document['x']]
    list.append(val)
df = pd.DataFrame(list, columns=['building_name', 'address_name','lat','lon'])
df
# 나온 결과들 중 필요한 녀석들('address_name','x','y')을 잡아 list형태로 만들어 DataFrame안에 넣어주자!

,building_name,address_name,lat,lon
0,,부산 동래구 사직동 26-9,35.1990136794377,129.059764973464


In [108]:
df = pd.read_excel('가맹점 위도경도 실패2.xlsx', sheet_name='Sheet1')
df

,Unnamed: 0,Unnamed: 1,location
0,황금성,051-505-8333,부산 동래구 중앙대로1277번길 124
1,동천홍,051-502-1240,부산 동래구 온천장로 1-1\n\n 온천동 473-12
2,아서원,010 85516716,부산 동래구 명장1동 명안로46번마길 15 15
3,평산옥,051-468-6255,부산 동구 초량중로 26
4,가덕도청솔식당,051-264-3454,부산 강서구 가덕해안로773번길 166
...,...,...,...
208,와분식,051-343-0203,부산광역시 북구 상리로 27 109호
209,해동반점,051-416-2250,부산 영도구 절영로 151
210,꽃곰탕,0507-1307-0919,부산광역시 동구 중앙대로361번길 37 1층
211,빈스킹토스트 부산진역점,051-442-7007,부산 동구 중앙대로371번길 9 1층\n\n 수정동 3-3


In [16]:
def address_to_latlon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json?query=" + address
    result = requests.get(urlparse(url).geturl(),
                      headers={"Authorization": "KakaoAK 073a393656181c6073880062d3507191"})
    json_obj = result.json()
    for document in json_obj['documents']:
        val = [document['address_name'], document['y'],document['x']]
    return val

#위의 두 녀석들을 합쳐 address가 입력되면 address_name, y, x를 반환하는 함수로 만들자.

In [114]:
address_to_latlon("부산 북구 금곡대로285번길 19 리버사이드빌딩 1층 101,102,201호")

['부산 북구 금곡대로285번길 19', '35.2337717376099', '129.012324342859']

In [109]:
list = []
for address in df['location']:
    list.append(address_to_latlon(address))
    
df1 = pd.DataFrame(list, columns = ['address_name', 'lat', 'lon'])
df1
# 받아온 엑셀파일(df) 속 '주소' column들을 for문으로 함수에 쏙쏙.

UnboundLocalError: local variable 'val' referenced before assignment

위의 함수를 통해 카카오맵 Documents에 요청할 수 있게 되었다.<br>
하지만 이 역시 Documents내에 해당 정보가 있을 때, 즉 입력한 값이 주소명일 때만 사용이 가능했다.<br>
따라서 카카오맵에서 가게명을 검색해 -> 주소를 받아온 후 -> 이를 다시 위도경도 변환 함수에 넣어줘야만 한다.

In [97]:
df1.to_excel('.~401.xlsx', sheet_name='1', index=False, header=False)
# to_excel함수를 통해 데이터프레임을 엑셀 파일로 저장한다.
# 이 때 함수에는 경로 및 파일이름 , 생성할 시트 이름 , index와 column값(False를 해두면 설정이 안된다)이 들어가야한다.